In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import glob
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

In [2]:
store=pd.read_csv('kakao_stores_0to500.csv',encoding='cp949')

In [3]:
review=pd.read_csv('kakao_review_0to500.csv',encoding='cp949')

---

In [4]:
recommand=store[['store_id','store_name', 'kakao_star_avg','addr','lon','lat']]

In [5]:
recommand=recommand[~(recommand['kakao_star_avg']=='후기미제공')]
recommand['kakao_star_avg']=recommand['kakao_star_avg'].astype('float64')
recommand=recommand[recommand['kakao_star_avg']>3]

In [6]:
recommand['category']=(store['cate_b']+' '+store['cate_c']).str.replace("/"," ")

review['kakao_content']+=' '
# recommand['location']=(store['lon'].astype(str)+' '+store['lat'].astype(str))

review_sum_list=[]
for store_id in recommand['store_id']:
    review_sum=review[(review['store_id']==store_id) & (~review['kakao_content'].isnull())]['kakao_content'].sum()
    if review_sum==0:
        review_sum=' '
    review_sum_list.append(review_sum)
recommand['review_sum']=review_sum_list

recommand['lon']=recommand['lon'].astype('str')
recommand['lat']=recommand['lat'].astype('str')

In [7]:
recommand.reset_index(drop=True,inplace=True)

In [8]:
len(recommand)

122

# 카테고리 벡터화/코사인 유사도 계산

In [9]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(recommand['category']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [10]:
#서로 카테고리 텍스트가 얼마나 유사한지를 따져줌

# 지역

In [11]:
count_vect_location = CountVectorizer(min_df=0, ngram_range=(1,2))

place_location = count_vect_location.fit_transform(recommand['addr']) 
place_simi_location = cosine_similarity(place_location, place_location) 
place_simi_location_sorted_ind = place_simi_location.argsort()[:, ::-1]

In [12]:
count_vect_lon = CountVectorizer(min_df=0, ngram_range=(1,2))

place_lon = count_vect_lon.fit_transform(recommand['lon']) 
place_simi_lon = cosine_similarity(place_lon, place_lon) 
place_simi_lon_sorted_ind = place_simi_lon.argsort()[:, ::-1]

In [13]:
count_vect_lat = CountVectorizer(min_df=0, ngram_range=(1,2))
place_lat = count_vect_lat.fit_transform(recommand['lat']) 
place_simi_lat = cosine_similarity(place_lat, place_lat) 
place_simi_lat_sorted_ind = place_simi_lat.argsort()[:, ::-1]

# 리뷰 텍스트 벡터화/ 코사인 유사도 계산

In [14]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(recommand['review_sum']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [15]:
place_simi_co = (
                 + place_simi_cate * 0.2 # 공식 1. 카테고리 유사도
                 + place_simi_lon * 0.35
                 + place_simi_lat * 0.35
                 +place_simi_location *0.5
                 + place_simi_review * 0.3 # 공식 2. 리뷰 텍스트 유사도
                 )

In [16]:
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 

In [17]:
def find_simi_place(df, sorted_ind, store_id, top_n=10):
    
    place_id = df[df['store_id'] == store_id]
    place_index = place_id.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [18]:
print(find_simi_place(recommand, place_simi_co_sorted_ind, 25882056, 5)[1:]['store_name'])

64      국수집
66    시골손만두
96    율동칼국수
54     조원냉면
Name: store_name, dtype: object


In [19]:
# 1. 로케이션에서 위도 경도 피쳐 따로 분리
# 2. 리뷰넘버빼기
# 3. 스타리뷰 빼기
# 4. 리뷰 널값 드랍

---

# 강사님이 주신 거

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs
import pickle
import pandas as pd

In [21]:
store_name_list=recommand['store_name'].to_list()

In [22]:
store_comment_list=recommand['review_sum'].to_list()

In [23]:
vectorizer = TfidfVectorizer(min_df=3,stop_words=['그냥','너무','거의'])

In [24]:
X = vectorizer.fit_transform(store_comment_list)
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['0점', '10년', '12000원', '2인분', '8천원', 'ㅋㅋ', 'ㅎㅎ', 'ㅠㅠ', '가게', '가게가',
       '가격', '가격대비', '가격도', '가격에', '가격은', '가격이', '가고', '가는', '가서', '가성비',
       '가요', '가장', '가족끼리', '간장', '감사합니다', '갑니다', '갔는데', '같아요', '같은데',
       '같이', '개인적으론', '겁나', '것도', '계속', '고기', '고기가', '고기는', '고기도', '고기를',
       '괜찮고', '괜찮았어요', '괜찮은', '괜찮음', '굉장히', '국물', '국물에', '국물이', '굳이',
       '굿굿', '그래서', '그런', '그런지', '그리고', '근데', '근처', '근처에', '기대', '기대하고',
       '기분', '김밥', '김치', '김치도', '깊은', '깍두기', '깔끔하고', '깔끔한', '나름', '나오고',
       '나오는', '나온', '나와서', '나중에', '냄새', '냄새가', '너무나', '너무너무', '느낌', '느낌이',
       '다들', '다른', '다시', '다시는', '다신', '다음에', '다음엔', '담백한', '더욱', '동네',
       '되는', '두번째', '들어가서', '듬뿍', '따로', '때문에', '라면', '리뷰', '마세요', '마음에',
       '막국수', '만두', '많고', '많아서', '많아요', '많은데', '많음', '많이', '맛과', '맛나고',
       '맛나요', '맛남', '맛도', '맛에', '맛은', '맛을', '맛이', '맛입니다', '맛있게', '맛있고',
       '맛있네요', '맛있는', '맛있는데', '맛있다', '맛있다고', '맛있습니다', '맛있어서', '맛있어요',
       '맛있어요ㅎㅎ', '맛있어용', '맛있었습니다', '맛있었어요', '맛있음', '맛집', '

In [25]:
store_sim = cosine_similarity(X)
store_sim[:3]

array([[1.        , 0.        , 0.19204051, 0.        , 0.        ,
        0.04140339, 0.13743471, 0.0445245 , 0.        , 0.02772527,
        0.19342135, 0.11282554, 0.0337696 , 0.03047925, 0.        ,
        0.        , 0.        , 0.20657992, 0.04739235, 0.07119141,
        0.07912795, 0.        , 0.22891335, 0.07140625, 0.08309768,
        0.18621595, 0.07516544, 0.        , 0.        , 0.17332916,
        0.        , 0.        , 0.07663107, 0.        , 0.        ,
        0.        , 0.        , 0.1252817 , 0.        , 0.        ,
        0.08111036, 0.07588067, 0.        , 0.0877112 , 0.12204123,
        0.        , 0.        , 0.15444807, 0.        , 0.12276391,
        0.06022665, 0.1089131 , 0.14313437, 0.04138957, 0.        ,
        0.11173297, 0.0930939 , 0.03923899, 0.        , 0.06216323,
        0.06667295, 0.16888593, 0.        , 0.09246032, 0.08205207,
        0.11420868, 0.02582928, 0.10688252, 0.13097115, 0.        ,
        0.        , 0.        , 0.05856248, 0.  

In [26]:
def similar_recommend_by_movie_id(store_id, n_recommand):
      store_index = store_id-1
      # enumerate 함수로 [(리스트 인덱스 0, 유사도 0), (리스트 인덱스 1, 유사도 1)...]의
      # 리스트를 만듭니다. 그 후 각 튜플의 두 번째 항목, 즉 유사도를 이용하여 내림차순 정렬합니다.
      # 이렇게 만든 리스트의 가장 앞 튜플의 첫 번째 항목이 영화 ID가 됩니다.
      similar_store = sorted(list(enumerate(store_sim[store_index])),key=lambda x:x[1], reverse=True)
      recommended=1
      print(f"--- '{store_name_list[store_id]}' 추천식당 Top {n_recommand} ---\n")
      for store_info in similar_store[1:n_recommand]:
            # 주어진 영화와 가장 비슷한 영화는 그 영화 자신이므로 출력 시 제외합니다.
            store_name= store_name_list[store_info[0]]
            print(f'\t추천영화 {recommended}순위: {store_name}')
            recommended+=1

In [27]:
similar_recommend_by_movie_id(0,5)

--- '독도해물알찜' 추천식당 Top 5 ---

	추천영화 1순위: 만석골칼국수
	추천영화 2순위: 사강정육점식당
	추천영화 3순위: 가마솥장수두부
	추천영화 4순위: 행복마을팥칼국수
